In [1]:
from PIL import Image, ImageEnhance
import glob
import random
from pascal_voc_writer import Writer
from tqdm import tqdm

In [2]:
backgrounds = glob.glob("backgrounds/*.jpg")
classes = {0: 'RED', 1: 'YELLOW', 2: 'GREEN'}
no_crops = {0: 10, 1: 4, 2: 4}
no_training_samples = 25000

In [3]:
img_counter = 0

In [4]:
for i in tqdm(range(no_training_samples)):
    img_filename = "models/images/{:0>6d}".format(img_counter)+".jpg"
    background = Image.open(random.choice(backgrounds))
    writer = Writer(img_filename, background.size[0], background.size[1])
    imgclass = random.randint(0,2)
    
    crop = Image.open("crops/"+classes[imgclass].lower()+"_crop_"+str(random.randint(1, no_crops[imgclass]))+".jpg")
        
    new_crop_width = random.randint(10, crop.size[0])
    new_crop_height = int(crop.size[1]*new_crop_width/crop.size[0])
    crop = crop.resize((new_crop_width, new_crop_height))
        
    new_crop_brightness = random.uniform(0.7, 1.5)
    enhancer = ImageEnhance.Brightness(crop)
    crop = enhancer.enhance(new_crop_brightness)
        
    xmin = random.randint(0, background.size[0] - crop.size[0])
    xmax = xmin + crop.size[0]
    ymin = random.randint(0, background.size[1] - crop.size[1])
    ymax = ymin + crop.size[1]
    background.paste(crop, (xmin, ymin))
        
    writer.addObject(classes[imgclass], xmin, ymin, xmax, ymax)
        
    background.save(img_filename)
    writer.save("models/annotations/xmls/{:0>6d}".format(img_counter)+".xml")
    img_counter += 1

100%|██████████| 25000/25000 [06:34<00:00, 63.32it/s]


In [5]:
with open("models/annotations/trainval.txt", "w") as f:
    f.write("\n".join(["{:0>6d}".format(i) for i in range(no_training_samples)]))

In [94]:
#background.show()